In [1]:
%load_ext watermark
%watermark

2017-05-19T17:17:54-04:00

CPython 3.6.0
IPython 5.3.0

compiler   : GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)
system     : Darwin
release    : 16.5.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit


In [108]:
from tf_idf import preprocess_fundamentos, remove_accents
from sklearn import model_selection
from tqdm import tqdm, trange
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter, defaultdict
from operator import itemgetter

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import PNUD
import logging
import random
import pandas as pd
import numpy as np

from pathlib import Path
from pprint import pprint

from sklearn.preprocessing import LabelEncoder
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC  # support vector machine classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB  # naive bayes
from sklearn.neighbors import KNeighborsClassifier

logger = logging.getLogger(__name__)

engine = create_engine('mysql://root@localhost/constabierta')
_session = sessionmaker(bind=engine, autocommit=True, expire_on_commit=True)
session = _session()

In [36]:
# load data
%time documents = session.query(PNUD).all()

# create pandas df
cols = ('id', 'id_ela', 'tema', 'categoria', 'pnud', 'concepto_original', 'fundamento', 'modo')
%time df = pd.DataFrame([[getattr(row, col) for col in cols] for row in documents], columns=cols)

CPU times: user 5.08 s, sys: 4.16 s, total: 9.24 s
Wall time: 13.1 s
CPU times: user 1.52 s, sys: 69.7 ms, total: 1.59 s
Wall time: 1.78 s


In [37]:
print("rows, columns:", df.shape)
df.head()

rows, columns: (178677, 8)


,id,id_ela,tema,categoria,pnud,concepto_original,fundamento,modo
0,4821,..nNa5GlUmV58fZt,1,Justicia,0,None,Que haya igualdad en el acceso a la justicia r...,Propositivo
1,4828,..opSItA3jigopqh,1,Justicia,0,None,una justicia mas adecuada a las necesidades de...,Propositivo
2,4835,..Y.aUt2kpvprcVi,1,Justicia,0,None,Que la justicia sea igual para todas la person...,Propositivo
3,4870,.1sARqGonxpcNy3L,1,Justicia,0,None,Todos los chilenos merecemos los mismos derech...,factico.definicion
4,4880,.2dhdwJZMOs8.bk6,1,Justicia,0,None,"En lo distributivo, dar a cada cual lo que le ...",propositivo


# Tarea 1: PNUD == 0



In [86]:
temas = range(1, 5)

pnud0 = df[df.pnud==0]
print("rows, columns:", pnud0.shape)
pnud0.head()

rows, columns: (169242, 8)


,id,id_ela,tema,categoria,pnud,concepto_original,fundamento,modo
0,4821,..nNa5GlUmV58fZt,1,Justicia,0,None,Que haya igualdad en el acceso a la justicia r...,Propositivo
1,4828,..opSItA3jigopqh,1,Justicia,0,None,una justicia mas adecuada a las necesidades de...,Propositivo
2,4835,..Y.aUt2kpvprcVi,1,Justicia,0,None,Que la justicia sea igual para todas la person...,Propositivo
3,4870,.1sARqGonxpcNy3L,1,Justicia,0,None,Todos los chilenos merecemos los mismos derech...,factico.definicion
4,4880,.2dhdwJZMOs8.bk6,1,Justicia,0,None,"En lo distributivo, dar a cada cual lo que le ...",propositivo


In [113]:
pnud0_temas = dict()
print("tema", "categs", "cantidad", sep="\t")
for tema in temas:
    pnud0_temas[tema] = pnud0[pnud0.tema == tema]
    print(tema, 
          len(pnud0_temas[tema]['categoria'].unique()), 
          pnud0_temas[tema].shape[0],
          sep='\t')
print()

for tema in temas:
    print("Categorias tema", tema)
    pprint(sorted(pnud0_temas[tema]['categoria'].unique().tolist()))
    print()

tema	categs	cantidad
1	37	46108
2	44	43704
3	12	40130
4	21	39300

Categorias tema 1
['Amistad cívica',
 'Autonomía / Libertad',
 'Bien Común / Comunidad',
 'Ciudadanía',
 'Democracia',
 'Desarrollo',
 'Descentralización',
 'Dignidad',
 'Diversidad',
 'Emprendimiento libre',
 'Equidad de género',
 'Estado de Derecho',
 'Estado laico',
 'Identidad cultural',
 'Igualdad',
 'Inclusión',
 'Innovación / Creatividad',
 'Integración',
 'Justicia',
 'Multiculturalidad',
 'Participación',
 'Patriotismo',
 'Paz / Convivencia pacífica',
 'Pluralismo',
 'Plurinacionalismo',
 'Probidad',
 'República',
 'Respeto',
 'Respeto / Conservación de la naturaleza o medio ambiente',
 'Responsabilidad',
 'Seguridad',
 'Soberanía',
 'Solidaridad',
 'Subsidiaridad',
 'Tolerancia',
 'Transparencia y publicidad',
 'Unidad']

Categorias tema 2
['A huelga',
 'A la educación',
 'A la honra / Al honor',
 'A la identidad cultural',
 'A la información',
 'A la integración de discapacidad',
 'A la integridad física y psí

## Crear train/test/dev sets por tema

Manteniendo la distribución de categorías

In [126]:
#### ejecutar esto para generar nuevos datasets de categorias
#### los archivos se guardan en la carpeta ./data/
#### el random seed define cómo se van a repartir los datos

SEED = 37

for tema in temas:
    fundamentos = pnud0[pnud0.tema == tema]['fundamento'].values.tolist()
    categorias = pnud0[pnud0.tema == tema]['categoria'].values.tolist()
    
    le = LabelEncoder()
    labels = le.fit_transform(categorias)

    X_train, X_testdev, y_train, y_testdev = model_selection.train_test_split(fundamentos, 
                                                                              labels, 
                                                                              test_size=0.2, 
                                                                              random_state=SEED,
                                                                              stratify=labels)
    
    X_test, X_dev, y_test, y_dev = model_selection.train_test_split(X_testdev,
                                                                    y_testdev, 
                                                                    test_size=0.5, 
                                                                    random_state=SEED, 
                                                                    stratify=y_testdev)
    
    print(f"tema_{tema}", "X", "y", sep="\t")
    print("train", len(X_train), len(y_train), sep="\t")
    print("test", len(X_test), len(y_test), sep="\t")
    print("dev", len(X_dev), len(y_dev), sep="\t")
    print()
    
    
    with Path(f'data/categorias_tema_{tema}_pnud_0.txt').open('w') as f:
        for line in le.classes_:
            f.write(str(line) + '\n')
    
    with Path(f'data/x_train_tema_{tema}_categorias_pnud_0.txt').open('w') as f:
        for line in X_train:
            f.write(line + '\n')
            
    with Path(f'data/y_train_tema_{tema}_categorias_pnud_0.txt').open('w') as f:
        for line in y_train:
            f.write(str(line) + '\n')
            
    with Path(f'data/x_test_tema_{tema}_categorias_pnud_0.txt').open('w') as f:
        for line in X_test:
            f.write(line + '\n')
            
    with Path(f'data/y_test_tema_{tema}_categorias_pnud_0.txt').open('w') as f:
        for line in y_test:
            f.write(str(line) + '\n')
            
    with Path(f'data/x_dev_tema_{tema}_categorias_pnud_0.txt').open('w') as f:
        for line in X_dev:
            f.write(line + '\n')
            
    with Path(f'data/y_dev_tema_{tema}_categorias_pnud_0.txt').open('w') as f:
        for line in y_dev:
            f.write(str(line) + '\n')

tema_1	X	y
train	36886	36886
test	4611	4611
dev	4611	4611

tema_2	X	y
train	34963	34963
test	4370	4370
dev	4371	4371

tema_3	X	y
train	32104	32104
test	4013	4013
dev	4013	4013

tema_4	X	y
train	31440	31440
test	3930	3930
dev	3930	3930



In [137]:
pprint(list(zip(X_train, le.inverse_transform(y_train)))[:10])

[('Para equilibrar los poderes del Estado, para que no exista solo una persona '
  'que decida todo.',
  'Congreso o parlamento'),
 ('Regula el gasto y cautela la probidad.',
  'Contraloría general / Tribunales de cuentas'),
 ('Poder judicial probo, ecuánime y               Equitativo, respeto absoluto '
  'a la ley.',
  'Poder judicial (estructura y funciones)'),
 ('Como institución fundamental de una elección democrática, sin reelección.',
  'Presidencia de la República'),
 ('El poder legislativo es fundamental en una república, por ende esta '
  'institución debe primar en la nueva constitución y su figura legislativa.',
  'Congreso o parlamento'),
 ('Es fundamental la autonomía de las regiones en su funcionamiento, para así '
  'descentralizar el país y beneficiarlos a todos.',
  'Gobierno regional'),
 ('el territorio soberano de Chile debe tener un criterio geográfico, político '
  'y administrativo de separación para una mayor distribución de los recursos.',
  'División territori